In [1]:
# Enforces PEP-8 style guideline
!pip install --upgrade pip
!pip install flake8 pycodestyle_magic

# Adds upper path to import common module
import sys
sys.path.append('../../')

/bin/bash: pip: command not found
/bin/bash: pip: command not found


In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [ ]:
"""
Version 1: TIMEOUT
- used methods: none

"""


# Returns unit matrix
def unitmatrix(n):
    return [[1 if i == j else 0 for j in range(n)] for i in range(n)]


# Returns multiplication of matrix; a * b
def matmul(a, b):
    l, m = len(a), len(a[0])
    n, p = len(b), len(b[0])
    # m == n
    if m != n:
        raise Exception

    # l * p
    c = [[0]*p for _ in range(l)]
    for i in range(l):
        for j in range(p):
            c[i][j] = sum(a[i][k] * b[k][j] for k in range(m))

    return c


# Returns power of matrix; m ^ n
def matpow(m, n):
    if n == 0:
        return unitmatrix(len(m))

    if n == 1:
        return m

    if n % 2 == 1:
        return matmul(m, matpow(m, n - 1))

    temp = matpow(m, n / 2)
    return matmul(temp, temp)


# Returns the probabilities that Dr. dunibal is in the towns.
def solution(adj_matrix, prison_town, days, suspicious_towns):

    # Process graph into probability matrix
    prob_matrix = []
    for row in adj_matrix:
        # All connected towns have equal chance to be selected
        num_connected_towns = row.count(1)
        prob_matrix.append([v / num_connected_towns for v in row])

    prob_matrix = matpow(prob_matrix, days)
    return [round(prob_matrix[prison_town][town], 8) for town in suspicious_towns]  # noqa: E501


C = int(input())
for _ in range(C):
    n, d, p = list(map(int, input().split()))
    adj_matrix = []
    for _ in range(n):
        row = list(map(int, input().split()))
        adj_matrix.append(row)

    _ = input()
    suspicious_towns = list(map(int, input().split()))
    result = solution(adj_matrix, p, d, suspicious_towns)
    print(' '.join(['{:.8f}'.format(prob) for prob in result]))

In [3]:
"""
Version 2:
- used methods: DP

"""
from functools import lru_cache


# Returns the probabilities that Dr. dunibal is in the towns.
def solution(adj_matrix, prison_town, days, suspicious_towns):
    n = len(adj_matrix)
    cnt_adjcents = [row.count(1) for row in adj_matrix]

    # Returns the probability of reaching :prison_town
    # in :remaining_days days
    @lru_cache(n*days)
    def solve(town, remaining_days):
        # Timed out
        if remaining_days == 0:
            return 1.0 if town == prison_town else 0.0

        ret = 0.0
        for next in range(n):
            if adj_matrix[town][next] == 1:
                # NOTICE: cnt_adjcents[next], not cnt_adjcents[town]
                ret += solve(next, remaining_days - 1) / cnt_adjcents[next]

        return ret

    return [round(solve(town, days), 8) for town in suspicious_towns]


C = int(input())
for _ in range(C):
    n, d, p = list(map(int, input().split()))
    adj_matrix = []
    for _ in range(n):
        row = list(map(int, input().split()))
        adj_matrix.append(row)

    _ = input()
    q = list(map(int, input().split()))
    result = solution(adj_matrix, p, d, q)
    print(' '.join(['{:.8f}'.format(prob) for prob in result]))

In [4]:
"""
Testing

"""
import helpers as hlpr


f = hlpr.clock(hlpr.timeout(seconds=10)(solution))
test_cases = [
    (([
        [0, 1, 1, 1, 0],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0]
    ], 0, 2, [0, 2, 4]),
     [0.83333333, 0.00000000, 0.16666667]),
    (([
        [0, 1, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 1],
        [0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0]
    ], 3, 2, [3, 1, 2, 6]),
     [0.43333333, 0.06666667, 0.06666667, 0.06666667]),
    (([
        [0, 1],
        [1, 0]
    ], 0, 1, [0, 1]),
     [0.0, 1.0]),
    (([
        [0 if i == j else 1 for j in range(100)] for i in range(100)
    ], 0, 100, [0]),
     [0.01])
]

passed, failed = [], []
for idx, tc in enumerate(test_cases, start=1):
    case, expected = tc
    result = None
    try:
        result = f(*case)
        assert result == expected, \
            'wants {} but got {}'.format(hlpr.truncate(expected, 20), hlpr.truncate(result, 20))  # noqa: E501
    except Exception as err:
        fmt = 'Case {} FAIL: {}'
        print(fmt.format(idx, str(err)))
        failed.append(idx)
    else:
        fmt = 'Case {} PASS'
        print(fmt.format(idx))
        passed.append(idx)

    print()

print()
if len(passed) == len(test_cases):
    print('All test(s) have passed')
else:
    fmt = '{} of {} test(s) failed: {}'
    print(fmt.format(len(failed), len(test_cases), ', '.join(map(str, failed))))  # noqa: E501

[0.00011992] 'solution([[0, 1, 1, 1, 0], [1, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0]], 0, 2, [0, 2, 4])' → [0.83333333, 0.0, 0.16666667]
Case 1 PASS

[0.00017762] 'solution([[0, 1, 1, 1, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1, 0, 0], [0, 0... → [0.43333333, 0.06666667, 0.06666667, 0.06666667]
Case 2 PASS

[0.00006557] 'solution([[0, 1], [1, 0]], 0, 1, [0, 1])' → [0.0, 1.0]
Case 3 PASS

[0.33944297] 'solution([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... → [0.01]
Case 4 PASS


All test(s) have passed


In [ ]:
!pip install matplotlib
"""
Performance analysis
- x-axis: n
- y-axis: O(f(n))

"""
import matplotlib as plt

_